## Project Overview 

###  Goals: To find drivers of churn.

Why? 

To help idenitfy and potentialy retain profitable customers who might drop Telco as thier communcaitions service provider.

In [ ]:
Data Preparation:
    I changed the data type for Total Charges because it was non-numerical.
    Also, the 
 